In [1]:
%load_ext autoreload
%autoreload 1
%aimport utils
%aimport ocfeats


ModuleNotFoundError: No module named 'utils'

In [ ]:
from pathlib import Path

import pandas as pd # also openpyxl for xlsx files
import numpy as np
import scipy.signal as ss
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.spatial import ConvexHull

from tqdm.auto import tqdm, trange

from utils import read_trc, read_mot
import ocfeats

# gdrive = Path('/Volumes/GoogleDrive-112026393729621442608')
gdrive = Path('/Users/psr/Library/CloudStorage/GoogleDrive-paru@stanford.edu')
datadir = gdrive / 'My Drive/NMBL Lab/OpenCap for NMD biomarkers/data'
dataset = '2023-05_dhd'

def get_trc_fpath(sid, trial):
    return datadir / dataset / f'opencap_data/{sid}/' \
        f'MarkerData/PostAugmentation/{trial}/{trial}.trc'

def get_mot_fpath(sid, trial):
    return datadir / dataset / f'opencap_data/{sid}/' \
        f'OpenSimData/Kinematics/{trial}.mot'

def get_model_path(sid):
    model_dir = datadir / dataset / f'opencap_data/{sid}/OpenSimData/Model/'
    return list(mdir.glob('*.osim'))[0]

    

In [ ]:
# colorblind friendly palette
from cycler import cycler
cp = ["#172A5A", "#FF7171", "#227567", "#34BAEA", "#F9D466", ]
plt.rcParams['axes.prop_cycle'] = cycler(color=cp)

# set default font
plt.rcParams['font.family'] = 'sans-serif'
plt.rcParams['font.sans-serif'] = ['Open Sans', 'Arial']

# automatically despine
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False

# set defualt DPI
plt.rcParams['figure.dpi'] = 72



In [ ]:
df_session = pd.read_excel(datadir / dataset / 'session_info.xlsx', )
df_trial = pd.read_excel(datadir / dataset / 'trial_info.xlsx')
df_part = pd.read_excel(datadir / dataset / 'participant_info.xlsx')


In [ ]:
df_temp = df_trial[df_trial.trial_clean == 'arm_rom']
df_temp = df_temp[['pid', 'sid', 'trial']]
um = df_temp.sample(1).iloc[0]
pid, sid, trial = um[['pid', 'sid', 'trial']]


In [ ]:
df = read_mot(get_mot_fpath(sid, trial))

plt.figure(figsize=(10,2))
plt.plot(df.time, df.arm_flex_r)
plt.plot(df.time, df.arm_flex_l)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10,2))
plt.plot(df.time, df.arm_add_r)
plt.plot(df.time, df.arm_add_l)
plt.show()

plt.figure(figsize=(10,2))
plt.plot(df.time, df.arm_rot_r)
plt.plot(df.time, df.arm_rot_l)
plt.tight_layout()
plt.show()

plt.figure(figsize=(10,2))
plt.plot(df.time, df.pro_sup_r)
plt.plot(df.time, df.pro_sup_l)
plt.tight_layout()
plt.show()


In [ ]:
pid = 'p014'
# pid = 'p060'
# pid = 'p093'

df_temp = df_trial[df_trial.trial_clean == 'arm_rom']
df_temp = df_temp[df_temp.pid == pid]
um = df_temp.sample(1).iloc[0]
pid, sid, trial = um[['pid', 'sid', 'trial']]
print(pid)

fps, markers, xyz = read_trc(get_trc_fpath(sid, trial))

rw = xyz[:,np.argmax(markers=='RWrist'),1]
lw = xyz[:,np.argmax(markers=='LWrist'),1]
mean_w = (rw + lw) / 2

rsa, rea, lsa, lea = ocfeats.trc_arm_angles(xyz, markers)

mean_sa = (rsa + lsa) / 2
mean_ea = (rea + lea) / 2

max_rsa = np.max(rsa)
max_lsa = np.max(lsa)

max_mean_sa = np.max(mean_sa)

rea_at_max_rsa = rea[np.argmax(rsa)]
lea_at_max_lsa = lea[np.argmax(lsa)]
mean_ea_at_max_mean_sa = mean_ea[np.argmax(mean_sa)]

rea_at_max_rw = rea[np.argmax(rw)]
lea_at_max_lw = lea[np.argmax(lw)]

max_min_sa = np.vstack([rsa, lsa]).min(0).max()

t = np.arange(xyz.shape[0]) / fps
plt.figure(figsize=(10,2))
plt.plot(t, rsa, lw=1)
plt.plot(t, rea)
plt.plot(t, lsa)
plt.plot(t, lea)
plt.axhline(mean_ea_at_max_mean_sa)
plt.axvline(t[np.argmax(mean_sa)])
plt.axvline(t[np.argmax(mean_w)])
plt.xlim(t[0], t[-1])
plt.ylim(0, 180)
plt.yticks([0, 45, 90, 135, 180])
plt.tight_layout()
plt.show()


In [ ]:
df_temp = df_trial[df_trial.trial_clean == 'arm_rom']
data = []
for i, row in tqdm(df_temp.iterrows(), total=df_temp.shape[0]):
    pid, sid, trial = row[['pid', 'sid', 'trial']]
    # print(pid)

    fps, markers, xyz = read_trc(get_trc_fpath(sid, trial))

    rw = xyz[:,np.argmax(markers=='RWrist'),1]
    lw = xyz[:,np.argmax(markers=='LWrist'),1]
    mean_w = (rw + lw) / 2

    rsa, rea, lsa, lea = ocfeats.trc_arm_angles(xyz, markers)

    mean_sa = (rsa + lsa) / 2
    mean_ea = (rea + lea) / 2

    max_rsa = np.max(rsa)
    max_lsa = np.max(lsa)

    max_mean_sa = np.max(mean_sa)

    rea_at_max_rsa = rea[np.argmax(rsa)]
    lea_at_max_lsa = lea[np.argmax(lsa)]
    mean_ea_at_max_mean_sa = mean_ea[np.argmax(mean_sa)]

    rea_at_max_rw = rea[np.argmax(rw)]
    lea_at_max_lw = lea[np.argmax(lw)]
    mean_ea_at_max_mean_w = mean_ea[np.argmax(mean_w)]
    
    max_min_sa = np.vstack([rsa, lsa]).min(0).max()
    
    data.append({'pid': pid,
                 # 'max_rsa': max_rsa,
                 # 'max_lsa': max_lsa,
                 'max_mean_sa': max_mean_sa,
                 'mean_ea_at_max_mean_sa': mean_ea_at_max_mean_sa,
                 'mean_ea_at_max_mean_w': mean_ea_at_max_mean_w,
                 'max_min_sa': max_min_sa,
                })


In [ ]:
df_arm_rom = pd.DataFrame(data)
df_temp = df_arm_rom.merge(df_part[['pid', 'brooke', 'type']],
                           on='pid', how='left')

plt.figure(figsize=(7,3))
sns.swarmplot(df_temp, x='type', y='max_mean_sa')
plt.xlabel('')
sns.despine()
plt.tight_layout()
plt.show()

plt.figure(figsize=(7,3))
sns.swarmplot(df_temp, x='type', y='mean_ea_at_max_mean_sa')
plt.xlabel('')
sns.despine()
plt.tight_layout()
plt.show()

plt.figure(figsize=(7,3))
sns.swarmplot(df_temp, x='type', y='mean_ea_at_max_mean_w')
plt.xlabel('')
sns.despine()
plt.tight_layout()
plt.show()

plt.figure(figsize=(7,3))
sns.swarmplot(df_temp, x='type', y='max_min_sa')
plt.xlabel('')
sns.despine()
plt.tight_layout()
plt.show()

sns.pairplot(df_temp, hue='type')
plt.show()


In [ ]:
df_temp = df_trial[df_trial.trial_clean == 'arm_rom']
data = []
for i, row in tqdm(df_temp.iterrows(), total=df_temp.shape[0]):
    pid, sid, trial = row[['pid', 'sid', 'trial']]
    # print(pid)

    fps, markers, xyz = read_trc(get_trc_fpath(sid, trial))

    rs = xyz[:,np.argmax(markers=='r_shoulder_study'),:]
    ls = xyz[:,np.argmax(markers=='L_shoulder_study'),:]
    rw = xyz[:,np.argmax(markers=='r_mwrist_study'),:]
    lw = xyz[:,np.argmax(markers=='L_mwrist_study'),:]

    rarm = rw-rs
    larm = lw-ls

    # arm length scaling
    # TODO use scaled model instead
    rarm /= np.median(np.linalg.norm(rarm, axis=1))
    larm /= np.median(np.linalg.norm(larm, axis=1))

    rrw = ConvexHull(rarm).volume
    lrw = ConvexHull(larm).volume
    rw = ConvexHull(np.concatenate([rarm, larm + ls-rs])).volume

    data.append({'pid': pid,
                 # 'max_rsa': max_rsa,
                 # 'max_lsa': max_lsa,
                 'right_reachable_workspace': rrw,
                 'left_reachable_workspace': lrw,
                 'reachable_workspace': rw,
                })


In [ ]:
df_arm_rom = pd.DataFrame(data)
df_temp = df_arm_rom.merge(df_part[['pid', 'brooke', 'type']],
                           on='pid', how='left')

plt.figure(figsize=(7,3))
sns.swarmplot(df_temp, x='type', y='right_reachable_workspace')
plt.xlabel('')
sns.despine()
plt.tight_layout()
plt.show()

plt.figure(figsize=(7,3))
sns.swarmplot(df_temp, x='type', y='left_reachable_workspace')
plt.xlabel('')
sns.despine()
plt.tight_layout()
plt.show()

plt.figure(figsize=(7,3))
sns.swarmplot(df_temp, x='type', y='reachable_workspace')
plt.xlabel('')
sns.despine()
plt.tight_layout()
plt.show()

sns.pairplot(df_temp, hue='type')
plt.show()


In [ ]:
print(sid)

model_path = get_model_path(sid)
print(model_path)
